# Retail Demo Store Messaging Workshop - Braze

In this workshop we will use [Braze](https://braze.com/) to add the ability to personalize marketing messages to customers of the Retail Demo Store using customer behavioral data and the Personalize models you trained in the prior workshops. We are going to define a campaign in Braze to send target users an email with product recommendations from the Amazon Personalize campaign we created in the Personalization workshop earlier.  You will also create an email template using the Braze Connected Content feature.  When the campaign is launched, Connected Content will send emails to the users that belong in the target group.  These emails will be formatted using the Personalize Campaign you created earlier.

Recommended Time: 1 hour

## Prerequisites

Since this module uses Amazon Personalize to generate and associate personalized product recommendations for users, it is assumed that you have either completed the [Personalization](../1-Personalization/1.1-Personalize.ipynb) workshop or those resources have been pre-provisioned in your AWS environment. If you are unsure and attending an AWS managed event such as a workshop, check with your event lead.

This part of the workshop will also require access to a Braze account.  If you are unsure of how to set that up conact <TODO: INSERT CONTACT HERE>.

## Architecture

Before diving into setting up Braze to send personalized messages to our users, let's review the relevant parts of the Retail Demo Store architecture and how it uses Braze to integrate with the machine learning campaigns created in Personalize.

[ Insert Images Here ]

### Data Collection via AWS Amplify

The Retail Demo Store's Web UI leverages [AWS Amplify](https://aws.amazon.com/amplify/) to integrate with AWS services for authentication ([Amazon Cognito](https://aws.amazon.com/cognito/)), messaging and analytics ([Amazon Pinpoint](https://aws.amazon.com/pinpoint/)), and to keep our personalization ML models up to date ([Amazon Personalize](https://aws.amazon.com/personalize/)). AWS Amplify provides libraries for JavaScript, iOS, Andriod, and React Native for building web and mobile applications. For this workshop, we'll be focusing on how user information and events from the Retail Demo Store's Web UI are sent to Pinpoint. This is depicted as **(1)** and **(2)** in the architecture above. We'll also show how the user information and events synchronized to Pinpoint are used to create and send personalized messages.

When a new user signs up for a Retail Demo Store account, views a product, adds a product to their cart, completes an order, and so on, the relevant function is called in `AnalyticsHandler.js` in the Retail Demo Store Web UI. The new user sign up event triggers a call to the `AnalyticsHandler.identify` function where user information from Cognito is used to [update an endpoint](https://docs.aws.amazon.com/pinpoint/latest/apireference/apps-application-id-endpoints.html) in Pinpoint. In Pinpoint, an endpoint represents a destination that you can send messages to, such as a mobile device, email address, or phone number.

```javascript
// Excerpt from src/web-ui/src/analytics/AnalyticsHandler.js

export const AnalyticsHandler = {
    identify(user) {
        Vue.prototype.$Amplify.Auth.currentAuthenticatedUser().then((cognitoUser) => {
            let endpoint = {
                userId: user.id,
                optOut: 'NONE',
                userAttributes: {
                    Username: [ user.username ],
                    ProfileEmail: [ user.email ],
                    FirstName: [ user.first_name ],
                    LastName: [ user.last_name ],
                    Gender: [ user.gender ],
                    Age: [ user.age.toString() ],
                    Persona: user.persona.split("_")
                }
            }

            if (user.addresses && user.addresses.length > 0) {
                let address = user.addresses[0]
                endpoint.location = {
                    City: address.city,
                    Country: address.country,
                    PostalCode: address.zipcode,
                    Region: address.state
                }
            }

            if (cognitoUser.attributes.email) {
                endpoint.address = cognitoUser.attributes.email
                endpoint.channelType = 'EMAIL'
                Amplify.Analytics.updateEndpoint(endpoint)
            }
        })
    }
}
```

Once an `EMAIL` endpoint is created for our user, we can update attributes on that endpoint based on actions the user takes in the web UI. For example, when the user adds an item to their shopping cart, we'll set the attribute `HasShoppingCart` to `true` to indicate that this endpoint has an active shopping cart. We can also set metrics such as the number of items in the endpoint's cart. As we'll see later, we can use these attributes when building Campaigns in Pinpoint to target endpoints based on their activity in the application.

```javascript
// Excerpt from src/web-ui/src/analytics/AnalyticsHandler.js
productAddedToCart(userId, cart, product, quantity, experimentCorrelationId) {
    Amplify.Analytics.updateEndpoint({
        attributes: {
            HasShoppingCart: ['true']
        },
        metrics: {
            ItemsInCart: cart.items.length
        }
    })
}
```

When the user completes an order, we send revenue tracking events to Pinpoint, as shown below, and also update endpoint attributes and metrics. We'll see how these events, attributes, and metrics be can used later in this workshop.

```javascript
// Excerpt from src/web-ui/src/analytics/AnalyticsHandler.js
orderCompleted(user, cart, order) {
    // ...
    for (var itemIdx in order.items) {
        let orderItem = order.items[itemIdx]

        Amplify.Analytics.record({
                name: '_monetization.purchase', 
                attributes: { 
                userId: user ? user.id : null,
                cartId: cart.id,
                orderId: order.id.toString(),
                _currency: 'USD',
                _product_id: orderItem.product_id
            },
            metrics: {
                _quantity: orderItem.quantity,
                _item_price: +orderItem.price.toFixed(2)
            }
        })
    }

    Amplify.Analytics.updateEndpoint({
        attributes: {
            HasShoppingCart: ['false'],
            HasCompletedOrder: ['true']
        },
        metrics: {
            ItemsInCart: 0
        }
    })
}
```

### Building a Braze Email Template

Before you can run a campaign in Braze, you will need to create an email template that uses Braze Connected Content to fetch Product recommendations for the users you will be emailing via the campaign.

1. Log in to the Braze [console](https://dashboard.braze.com/auth).  You will need to have a Braze account configured for you to continue this workshop, or use your existing Braze account.

![Braze Login Screen](images/braze-create-template-0.png)

2. On the left hand pane of the Braze console, click the Templates & Media link.  This will show the Templates list screen.  Click the From File button under Basic Email Templates at the top of the screen.

![](images/braze-create-template-1.png)

3. A sample HTML template file has been provided for you in this workshop, to save editing time.  You can edit this template as needed but it is recommended that you follow this workshop at least once to faimilarize yourself with how this integration will work.  Click the Upload from File button:

![](images/braze-create-template-2.png)

4. Navigate to your local clone of this repository, to the `/workshop/4-Messaging/` folder.  Select the `braze-connected-content-email-template.zip` file and click the Open or Save button to upload the ZIP file to Braze

![](images/braze-create-template-3.png)

5. Braze will validate the file, and return a status window.  Click the Build Email button to continue

![](images/braze-create-template-4.png)

6. On the next screen, give the template a name, such as `Braze Connected Content Workshop Template`

![](images/braze-create-template-5.png)

7. On the bottom right of the screen, click the Save Template button.  Make sure to click the Save button often, the editor will not auto-save your changes.

![](images/braze-create-template-6.png)

8. Scroll back up and click the Edit Email Body link

![](images/braze-create-template-7.png)

9. This will take you to the HTML template editor.  At the very top, there is a Connected Content call that calls the Personalize recommendation service you deployed eariler, and gets a set of recommendations for the user ID that this template will be sent to:

```
{% connected_content <URL OF YOUR RECOMMENDATION SERVICE>/recommendations?userID={{${user_id}}}&fullyQualifyImageUrls=1&numResults=4 :save result %}
```

You will need to look up the URL for the deployment of your Personalize catalog recommendation service.  Once you have that, you will need to paste the following code into the section above:

```
http://<YOUR RECOMMENDATION SERVICE URL>/recommendations?userID={{${user_id}}}&fullyQualifyImageUrls=1&numResults=4
```

Do not change the URL query parameters; this will break the template in the next few steps.

This snippet will be processed by Braze when you run your campaign, and Connected Content will query your Personalize recommendation service with the `user_id` this email is intended for, fetch recommendations, and then will fill in the appropriate sections of the HTML template with information about recommended products.

![](images/braze-create-template-8.png)

10. Note that this template is hard-coded to show 4 product recommendations.  Once Connected Content loads recommendations, they are available in the `result` array and they can be referenced in the HTML for your template.

When you are finished modifying your template, click the Done button at the bottom right of the editor screen.

![](images/braze-create-template-9.png)

11. Before navigating away from the template editor, make sure to click the Save Template button on the bottom right of the template window, or your earlier changes will be lost.

![](images/braze-create-template-10.png)


### Verify Machine Learning Model / Recommender

If you open a web browser window/tab and browse to the Pinpoint service in the AWS console for the AWS account we're working with, you should see the ML Model / Recommender that we just created in Pinpoint.

![Pinpoint ML Model / Recommender](images/pinpoint-ml-model.png)

### Upload Braze User Profiles

So that we can send an email to more than one endpoint in this workshop, verify a couple more variations of your email address.

Assuming your **valid** email address is `joe@example.com`, add a few more variations using `+` notation such as...

- `joe+1@example.com`
- `joe+2@example.com`
- `joe+3@example.com`

Just enter a variation, click the "Verify email address" button, and repeat until you've added a few more. Write down or commit to memory the variations you created--we'll need them later.

By adding these variations, we're able to create separate Retail Demo Store accounts for each email address and therefore separate endpoints in Pinpoint that we can target. Note that emails sent to the these variations should still be delivered to your same inbox.

### Create a Braze Campaign



## Workshop Complete

Congratulations! You have completed the Retail Demo Store Braze Workshop.

### Cleanup

If you launched the Retail Demo Store in your personal AWS account **AND** you're done with all workshops & your evaluation of the Retail Demo Store, you can remove all provisioned AWS resources and data by deleting the CloudFormation stack you used to deploy the Retail Demo Store.

> If you are participating in an AWS managed event such as a workshop and using an AWS provided temporary account, you can skip the following cleanup steps unless otherwise instructed.